# Celebrity Mentions
*Example Usage of Scraipe Library*

Here's a quick example using scraipe to extract mentions of celebrities in news articles.

## Setup
Install and import things we need. We will use `NewsScraper` and `OpenAiAnalyzer` from the `scraipe[extended]` subpackage.

We will also load [your OpenAI API key](https://help.openai.com/en/articles/4936850-where-do-i-find-my-openai-api-key) from a file called 'openai_key.txt'. This file should be in the same folder as this notebook.

In [15]:
# Install Scraipe
%pip install scraipe[extended] --quiet --upgrade
# Check package version
!pip show scraipe | grep Version

Note: you may need to restart the kernel to use updated packages.
Version: 0.1.26


In [1]:

# Import modules
import pandas as pd
from scraipe import Workflow
from scraipe.extended import NewsScraper, OpenAiAnalyzer
from pydantic import BaseModel

# Load OpenAI API key
OPENAI_API_KEY = open("openai_key.txt").read().strip()

## Extract links
First, we need a list of links to target with scraipe. We will extract all links from the front page of https://apnews.com.

In [2]:
import requests
import re

url = "https://apnews.com/"
response = requests.get(url)
html = response.text

# Use a regex to extract article links
pattern = r'href=["\'](?=[^"\']*/article)([^"\']+)["\']'
news_links = re.findall(pattern, html)

# Remove duplicates
news_links = list(set(news_links))

# Display a summary of the links
news_links_df = pd.DataFrame(news_links, columns=['link'])
import time
print(f"Found {len(news_links_df)} front page AP News links on {time.strftime('%Y-%m-%d')}")
display(news_links_df.head())

Found 146 front page AP News links on 2025-04-05


,link
0,https://apnews.com/article/russia-ukraine-war-...
1,https://apnews.com/article/china-us-tariffs-ti...
2,https://apnews.com/article/sylvester-turner-co...
3,https://apnews.com/article/trump-deportations-...
4,https://apnews.com/article/measles-outbreak-te...


## Configure Workflow
Now we'll configure the scraipe workflow using `NewsScraper` and `OpenAiAnalyzer`.

`NewsScraper` uses `trafilatura` to extract article content from a news site without all the HTML clutter.

`OpenAiAnalyzer` uses OpenAI models to extract data from the article content.

In [3]:
#===Configure NewsScraper===
# NewsScraper doesn't require any additional configuration
scraper = NewsScraper()

#===Configure OpenAiAnalyzer===
# Define the instruction for the LLM. Ensure the instruction specifies a return schema.
instruction = '''
Extract a list of celebrities mentioned in the article text.
Return a JSON dictionary with the following schema:
{"celebrities":["celebrity1", "celebrity2", ...]}
'''

# (Optional) Create a pydantic schema to validate the LLM output
from typing import List
class ExpectedOutput(BaseModel):
    celebrities: List[str]
    
# Create the analyzer with the API key, instruction, and schema
analyzer = OpenAiAnalyzer(OPENAI_API_KEY,instruction,pydantic_schema=ExpectedOutput)

#===Create Workflow===
# Create a workflow with the configured scraper and analyzer
workflow = Workflow(scraper, analyzer)

## Scrape links
Next we will scrape content from news links. This content will be saved within the workflow's scrape store.

In [4]:
# Scrape the news links
workflow.scrape(news_links)
# Display the scraped content
display(workflow.get_scrapes().head())

Scraping: 100%|██████████| 146/146 [00:45<00:00,  3.20link/s]


,link,content,scrape_success,scrape_error
0,https://apnews.com/article/china-movies-imax-u...,Living abroad and hungry to see a Chinese movi...,True,None
1,https://apnews.com/article/trump-netanyahu-vis...,Trump’s upcoming White House meeting with Neta...,True,None
2,https://apnews.com/article/tortoise-babies-phi...,Galapagos tortoises at Philadelphia Zoo become...,True,None
3,https://apnews.com/article/argentina-milei-tru...,Milei refuerza su amistad con Trump mientras e...,True,None
4,https://apnews.com/article/tunisia-snails-alte...,"In Tunisia, snails inch toward replacing red m...",True,None


## Analyze content
Next we will extract celebrities mentioned in each article using OpenAI

In [5]:
# Analyze the scraped content
workflow.analyze()

# Display the analyses
display(workflow.get_analyses().head())

Analyzing: 100%|██████████| 146/146 [00:51<00:00,  2.82item/s]


,link,output,analysis_success,analysis_error
0,https://apnews.com/article/china-movies-imax-u...,{'celebrities': ['Donald Trump']},True,None
1,https://apnews.com/article/trump-netanyahu-vis...,"{'celebrities': ['Donald Trump', 'Benjamin Net...",True,None
2,https://apnews.com/article/tortoise-babies-phi...,{'celebrities': []},True,None
3,https://apnews.com/article/argentina-milei-tru...,"{'celebrities': ['Donald Trump', 'Javier Milei...",True,None
4,https://apnews.com/article/tunisia-snails-alte...,{'celebrities': []},True,None


## Compile the results
Finally, let's export the completed analysis. 

In [6]:
export_df = workflow.export()
display(export_df)
export_df.to_csv('celebrities.csv', index=False)

,link,scrape_success,analysis_success,celebrities
0,https://apnews.com/article/china-movies-imax-u...,True,True,[Donald Trump]
1,https://apnews.com/article/trump-netanyahu-vis...,True,True,"[Donald Trump, Benjamin Netanyahu]"
2,https://apnews.com/article/tortoise-babies-phi...,True,True,[]
3,https://apnews.com/article/argentina-milei-tru...,True,True,"[Donald Trump, Javier Milei, Elon Musk, Cristi..."
4,https://apnews.com/article/tunisia-snails-alte...,True,True,[]
...,...,...,...,...
141,https://apnews.com/article/2025-ncaa-march-mad...,True,True,[Haley Van Lith]
142,https://apnews.com/article/trump-deportations-...,True,True,"[Trump, Barack Obama]"
143,https://apnews.com/article/wrongful-conviction...,True,True,"[Donald Trump, Fani Willis, Nathan Wade, Brian..."
144,https://apnews.com/article/iran-rial-currency-...,True,True,"[Donald Trump, Ayatollah Ali Khamenei, Mehdi D..."


## Analyze the results
Now you can conduct your own analysis on the structured data collected by the scraipe workflow.

In [7]:
# Load the extracted data
celebrities_df = pd.read_csv('celebrities.csv')
from ast import literal_eval
celebrities_df['celebrities'] = celebrities_df['celebrities'].apply(literal_eval)

# Explode the nested list of celebrities
celebrities_df = celebrities_df.explode('celebrities')
celebrities_df['celebrities'] = celebrities_df['celebrities'].str.strip()

# Display the top 10 most mentioned celebrities
celebrities_df = celebrities_df['celebrities'].value_counts().reset_index()
celebrities_df.columns = ['celebrity', 'mentions']
celebrities_df = celebrities_df.sort_values('mentions', ascending=False)
celebrities_df.head(10)

,celebrity,mentions
0,Donald Trump,55
1,Elon Musk,17
2,Joe Biden,9
3,Robert F. Kennedy Jr.,4
4,Geno Auriemma,3
5,Wayne Gretzky,3
6,Alex Ovechkin,3
7,Bill Gates,3
9,Barack Obama,3
10,Chuck Schumer,3
